## Heart Disease Risk Prediction: Logistic Regression 
## Step 1: Load and prepare the Dataset

The hearth disease is one of the most important problems in the heathly area, that situation create the neccessity to search a solution or tratements for this problem, for that reason in kaggle exist information about predition or analizys of hearth desease. In this lab, i'm gonna explain this information and create an analysis with logistic regression.

In the first section focuses on: Prepare the dataset and exploring the data analysis.

## Import required libraries 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pit

np.random.seed(42)

# Load the Dataset

In [ ]:
df = pd.read_csv("heath.csv")
df.head()

# Dataset Overview

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()